# if you want to unzip the zip file through code

In [2]:
# import zipfile
# import os

# zip_path = r"C:\Users\saroj\Downloads\archive (1).zip"  # Change if your file name is different
# extract_path = "UTKFace"

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# print("Dataset extracted to", extract_path)


Dataset extracted to UTKFace


# IF you want to check the sub folders of UTK face Dataset

In [6]:
# import os

# image_dir = "UTKFace"  # Make sure this matches the name of your extracted folder

# # List files to confirm they exist and are in the right format
# if os.path.exists(image_dir):
#     files = os.listdir(image_dir)
#     print(f"Total files found: {len(files)}")
#     print("Sample files:", files[:5])
# else:
#     print("Directory not found. Check the path:", image_dir)


Total files found: 3
Sample files: ['crop_part1', 'UTKFace', 'utkface_aligned_cropped']


# Total Files inside the subfolders

In [7]:
# import os

# parent_dir = "UTKFace"  # or wherever these 3 folders are

# # Check subfolders
# subfolders = ['crop_part1', 'UTKFace', 'utkface_aligned_cropped']
# for folder in subfolders:
#     folder_path = os.path.join(parent_dir, folder)
#     if os.path.isdir(folder_path):
#         files = os.listdir(folder_path)
#         print(f"Folder: {folder}, Total files: {len(files)}")
#         print("Sample files:", files[:5])


Folder: crop_part1, Total files: 9780
Sample files: ['100_1_0_20170110183726390.jpg.chip.jpg', '100_1_2_20170105174847679.jpg.chip.jpg', '101_1_2_20170105174739309.jpg.chip.jpg', '10_0_0_20161220222308131.jpg.chip.jpg', '10_0_0_20170103200329407.jpg.chip.jpg']
Folder: UTKFace, Total files: 23708
Sample files: ['100_0_0_20170112213500903.jpg.chip.jpg', '100_0_0_20170112215240346.jpg.chip.jpg', '100_1_0_20170110183726390.jpg.chip.jpg', '100_1_0_20170112213001988.jpg.chip.jpg', '100_1_0_20170112213303693.jpg.chip.jpg']
Folder: utkface_aligned_cropped, Total files: 2
Sample files: ['crop_part1', 'UTKFace']


# Process the data for training

In [8]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

image_dir = "UTKFace/UTKFace"  # This is the correct path

data = []
age_labels = []
gender_labels = []

for filename in os.listdir(image_dir):
    try:
        if not filename.endswith(".jpg"):
            continue

        age, gender, *_ = filename.split("_")
        img_path = os.path.join(image_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.resize(img, (64, 64))

        data.append(img)
        age_labels.append(int(age))
        gender_labels.append(int(gender))
    except Exception as e:
        print(f"Skipping {filename}: {e}")
        continue

print(f"✅ Total valid images loaded: {len(data)}")

X = np.array(data) / 255.0
y_age = np.array(age_labels)
y_gender = np.array(gender_labels)

# Split into train/test
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(
    X, y_age, y_gender, test_size=0.2, random_state=42
)

print("✅ Data preprocessed and ready for model training.")


✅ Total valid images loaded: 23708
✅ Data preprocessed and ready for model training.


# Define Model

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Input Layer
input_layer = Input(shape=(64, 64, 3))

# Shared Convolutional Base
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2))(x)
x = BatchNormalization()(x)

x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Output 1: Gender Classification
gender_output = Dense(1, activation='sigmoid', name='gender_output')(x)

# Output 2: Age Regression
age_output = Dense(1, activation='linear', name='age_output')(x)

# Define Model
model = Model(inputs=input_layer, outputs=[gender_output, age_output])

# Compile Model
model.compile(
    optimizer='adam',
    loss={
        'gender_output': 'binary_crossentropy',
        'age_output': 'mae'
    },
    metrics={
        'gender_output': 'accuracy',
        'age_output': 'mae'
    }
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 64, 64, 3)         │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 64, 64, 32)        │             896 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 32, 32, 32)        │               0 │ conv2d[0][0]               │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 32, 32, 32)        │             128 │ max_pooling2d[0][0]        │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 32, 32, 64)        │          18,496 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_1               │ (None, 16, 16, 64)        │               0 │ conv2d_1[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 16, 16, 64)        │             256 │ max_pooling2d_1[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 16, 16, 128)       │          73,856 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d_2               │ (None, 8, 8, 128)         │               0 │ conv2d_2[0][0]             │
│ (MaxPooling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 8, 8, 128)         │             512 │ max_pooling2d_2[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 8192)              │               0 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │       1,048,704 │ flatten[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gender_output (Dense)         │ (None, 1)                 │             129 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ age_output (Dense)            │ (None, 1)                 │             12

 Total params: 1,143,106 (4.36 MB)

 Trainable params: 1,142,658 (4.36 MB)

 Non-trainable params: 448 (1.75 KB)

# Train Model

In [10]:
history = model.fit(
    X_train,
    {'gender_output': y_gender_train, 'age_output': y_age_train},
    validation_split=0.1,
    epochs=20,
    batch_size=64
)


Epoch 1/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 43s 148ms/step - age_output_loss: 13.2441 - age_output_mae: 13.2441 - gender_output_accuracy: 0.7209 - gender_output_loss: 1.0242 - loss: 14.2682 - val_age_output_loss: 14.3709 - val_age_output_mae: 14.3692 - val_gender_output_accuracy: 0.7343 - val_gender_output_loss: 0.5254 - val_loss: 14.8947
Epoch 2/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 39s 145ms/step - age_output_loss: 9.3611 - age_output_mae: 9.3611 - gender_output_accuracy: 0.8120 - gender_output_loss: 0.4883 - loss: 9.8494 - val_age_output_loss: 8.9158 - val_age_output_mae: 8.9092 - val_gender_output_accuracy: 0.8366 - val_gender_output_loss: 0.3648 - val_loss: 9.2756
Epoch 3/20
267/267 ━━━━━━━━━━━━━━━━━━━━ 39s 144ms/step - age_output_loss: 8.6566 - age_output_mae: 8.6566 - gender_output_accuracy: 0.8221 - gender_output_loss: 0.3965 - loss: 9.0530 - val_age_output_loss: 7.8469 - val_age_output_mae: 7.8429 - val_gender_output_accuracy: 0.8487 - val_gender_output_loss: 0.3237 - val_loss: 8.1675
E

# Summary results

In [12]:
results = model.evaluate(X_test, {'gender_output': y_gender_test, 'age_output': y_age_test})
print("\nTest Loss & Metrics:")
for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.4f}")


149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - age_output_loss: 6.4738 - age_output_mae: 6.4738 - gender_output_accuracy: 0.8820 - gender_output_loss: 0.2732 - loss: 6.7470

Test Loss & Metrics:
loss: 6.7535
compile_metrics: 0.2555
gender_output_loss: 6.4960
age_output_loss: 6.4972


In [13]:
import numpy as np

# Predict gender and age on test data
gender_pred_prob, age_pred = model.predict(X_test)

# Since gender is sigmoid output, convert to binary labels
gender_pred = (gender_pred_prob > 0.5).astype(int).reshape(-1)


149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step


In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

print("Gender Classification Metrics:")

cm = confusion_matrix(y_gender_test, gender_pred)
print("Confusion Matrix:\n", cm)

acc = accuracy_score(y_gender_test, gender_pred)
prec = precision_score(y_gender_test, gender_pred)
rec = recall_score(y_gender_test, gender_pred)
f1 = f1_score(y_gender_test, gender_pred)

print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1 Score: {f1:.4f}")


Gender Classification Metrics:
Confusion Matrix:
 [[2201  284]
 [ 256 2001]]
Accuracy: 0.8861
Precision: 0.8757
Recall: 0.8866
F1 Score: 0.8811


In [15]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print("\nAge Regression Metrics:")

r2 = r2_score(y_age_test, age_pred)
mae = mean_absolute_error(y_age_test, age_pred)
mse = mean_squared_error(y_age_test, age_pred)

print(f"R² Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")



Age Regression Metrics:
R² Score: 0.7940
MAE: 6.4972
MSE: 81.7811


In [16]:
# model.save("age_gender_model.h5")
# print("Model saved as age_gender_model.h5")


Model saved as age_gender_model.h5


In [19]:
# from tensorflow.keras.models import load_model
# from tensorflow.keras.metrics import MeanAbsoluteError

# model = load_model('age_gender_model.h5', custom_objects={'mae': MeanAbsoluteError()})


In [26]:
# from tensorflow.keras.losses import MeanAbsoluteError

# model.compile(
#     optimizer='adam',
#     loss={
#         'gender_output': 'binary_crossentropy',
#         'age_output': MeanAbsoluteError()
#     },
#     metrics={
#         'gender_output': 'accuracy',
#         'age_output': MeanAbsoluteError()
#     }
# )


In [24]:
# model.evaluate(X_test, {'gender_output': y_gender_test, 'age_output': y_age_test})


149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - age_output_loss: 6.4738 - age_output_mean_absolute_error: 6.4738 - gender_output_accuracy: 0.8820 - gender_output_loss: 0.2732 - loss: 6.7470


[6.753493785858154,
 0.25550103187561035,
 6.496010780334473,
 6.496010780334473,
 0.8861240148544312]

In [27]:
# import cv2
# import numpy as np
# from tensorflow.keras.models import load_model

# # Load your trained model
# model = load_model(r"C:\Users\saroj\Desktop\01\age_gender_model.h5")

# # Load OpenCV's pretrained Haar Cascade face detector
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# # Open webcam
# cap = cv2.VideoCapture(0)

# def preprocess_face(face_img):
#     face_img = cv2.resize(face_img, (64, 64))
#     face_img = face_img.astype('float32') / 255.0
#     face_img = np.expand_dims(face_img, axis=0)  # Shape: (1, 64, 64, 3)
#     return face_img

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     # Detect faces (can adjust scaleFactor and minNeighbors for tuning)
#     faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

#     # Only process if at least 3 faces detected
#     if len(faces) >= 3:
#         for (x, y, w, h) in faces:
#             # Extract face ROI from original color frame
#             face_img = frame[y:y+h, x:x+w]

#             # Preprocess and predict
#             input_img = preprocess_face(face_img)
#             gender_pred_prob, age_pred = model.predict(input_img)

#             gender_label = "Female" if gender_pred_prob[0][0] > 0.5 else "Male"
#             age_label = int(age_pred[0][0])

#             # Draw rectangle around face
#             cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

#             # Put text: gender and age
#             cv2.putText(frame, f"{gender_label}, Age: {age_label}", (x, y - 10),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

#     # Show the frame
#     cv2.imshow('Age & Gender Prediction', frame)

#     # Press 'q' to quit
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


TypeError: Could not locate function 'mae'. Make sure custom classes are decorated with `@keras.saving.register_keras_serializable()`. Full object config: {'module': 'keras.metrics', 'class_name': 'function', 'config': 'mae', 'registered_name': 'mae'}

# It will save the tarained model in .keras

In [30]:
model.save('age_gender_model1.keras', include_optimizer=False)


In [6]:
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError

model = load_model('age_gender_model1.keras', compile=False)

model.compile(
    optimizer='adam',
    loss={
        'gender_output': 'binary_crossentropy',
        'age_output': MeanAbsoluteError()
    },
    metrics={
        'gender_output': 'accuracy',
        'age_output': MeanAbsoluteError()
    }
)


# Demo code (don't run)

In [33]:
# import cv2
# import numpy as np
# from tensorflow.keras.models import load_model

# # Load your trained model
# model = load_model(r"C:\Users\saroj\Desktop\01\age_gender_model1.keras")

# # Load OpenCV's pretrained Haar Cascade face detector
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# # Open webcam
# cap = cv2.VideoCapture(0)

# def preprocess_face(face_img):
#     face_img = cv2.resize(face_img, (64, 64))
#     face_img = face_img.astype('float32') / 255.0
#     face_img = np.expand_dims(face_img, axis=0)  # Shape: (1, 64, 64, 3)
#     return face_img

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     # Detect faces (can adjust scaleFactor and minNeighbors for tuning)
#     faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

#     # Only process if at least 3 faces detected
#     if len(faces) >= 3:
#         for (x, y, w, h) in faces:
#             # Extract face ROI from original color frame
#             face_img = frame[y:y+h, x:x+w]

#             # Preprocess and predict
#             input_img = preprocess_face(face_img)
#             gender_pred_prob, age_pred = model.predict(input_img)

#             gender_label = "Female" if gender_pred_prob[0][0] > 0.5 else "Male"
#             age_label = int(age_pred[0][0])

#             # Draw rectangle around face
#             cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

#             # Put text: gender and age
#             cv2.putText(frame, f"{gender_label}, Age: {age_label}", (x, y - 10),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

#     # Show the frame
#     cv2.imshow('Age & Gender Prediction', frame)

#     # Press 'q' to quit
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()


C:\Users\saroj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 38 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


# This tells the predicted age
## If you run this sit with some distance having enough light in the room

In [3]:
# import cv2
# import numpy as np
# from tensorflow.keras.models import load_model
# from tensorflow.keras.losses import MeanAbsoluteError

# # Load the trained model and compile explicitly
# model = load_model('age_gender_model1.keras', compile=False)
# model.compile(
#     optimizer='adam',
#     loss={
#         'gender_output': 'binary_crossentropy',
#         'age_output': MeanAbsoluteError()
#     },
#     metrics={
#         'gender_output': 'accuracy',
#         'age_output': MeanAbsoluteError()
#     }
# )

# # Load Haar Cascade for face detection
# face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# def preprocess_face(face_img):
#     """Resize, normalize, and reshape face image for prediction."""
#     face_img = cv2.resize(face_img, (64, 64))
#     face_img = face_img.astype('float32') / 255.0
#     face_img = np.expand_dims(face_img, axis=0)  # Shape: (1, 64, 64, 3)
#     return face_img

# # Start webcam capture
# cap = cv2.VideoCapture(0)

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break

#     # Convert to grayscale for face detection
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     # Detect faces
#     faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

#     for (x, y, w, h) in faces:
#         # Extract the face ROI from original frame
#         face_img = frame[y:y+h, x:x+w]

#         # Preprocess for model
#         input_img = preprocess_face(face_img)

#         # Predict gender and age
#         gender_pred_prob, age_pred = model.predict(input_img)
#         gender_label = "Female" if gender_pred_prob[0][0] > 0.5 else "Male"
#         age_label = int(age_pred[0][0])

#         # Draw bounding box on the face
#         cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

#         # Text position above the box
#         text = f"{gender_label}, Age: {age_label}"
#         (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
#         cv2.rectangle(frame, (x, y - text_height - 10), (x + text_width, y), (0, 255, 0), -1)  # Filled box for text
#         cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

#     # Display the frame
#     cv2.imshow('Age & Gender Prediction', frame)

#     # Press 'q' to quit
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Cleanup
# cap.release()
# cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


# This is the code after trainig its predict the age and gender  ( child teen adult......)
## look straight towards the camera with good lighting

In [9]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError

# Load your trained multi-output model (.keras format)
model = load_model('age_gender_model1.keras', compile=False)
model.compile(
    optimizer='adam',
    loss={
        'gender_output': 'binary_crossentropy',
        'age_output': MeanAbsoluteError()
    },
    metrics={
        'gender_output': 'accuracy',
        'age_output': MeanAbsoluteError()
    }
)

# Haar cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def preprocess_face(face_img):
    """Resize and normalize face image for prediction."""
    face_img = cv2.resize(face_img, (64, 64))
    face_img = face_img.astype('float32') / 255.0
    face_img = np.expand_dims(face_img, axis=0)  # (1, 64, 64, 3)
    return face_img

def age_group(age):
    """Convert numeric age to categorical age group."""
    if age < 12:
        return "Kid"
    elif age < 36:
        return "Young Adult"
    elif age < 60:
        return "Middle-Aged"
    else:
        return "Senior Citizen"

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]

        input_img = preprocess_face(face_img)

        # Predict gender and age
        gender_pred_prob, age_pred = model.predict(input_img)
        gender_label = "Female" if gender_pred_prob[0][0] > 0.5 else "Male"
        age_label = age_group(int(age_pred[0][0]))

        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Prepare text label
        text = f"{gender_label}, {age_label}"

        # Calculate text size and draw filled rectangle for better readability
        (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)
        cv2.rectangle(frame, (x, y - text_height - 10), (x + text_width, y), (0, 255, 0), -1)

        # Put text above face box
        cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

    # Show frame
    cv2.imshow('Age Group & Gender Prediction', frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━

# This is just an demo UI don't refer this

In [5]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanAbsoluteError
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import threading
import time
import os

# Load model
model = load_model('age_gender_model1.keras', compile=False)
model.compile(
    optimizer='adam',
    loss={
        'gender_output': 'binary_crossentropy',
        'age_output': MeanAbsoluteError()
    },
    metrics={
        'gender_output': 'accuracy',
        'age_output': MeanAbsoluteError()
    }
)

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def preprocess_face(face_img):
    face_img = cv2.resize(face_img, (64, 64))
    face_img = face_img.astype('float32') / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    return face_img

def age_group(age):
    if age < 12:
        return "Kid"
    elif age < 36:
        return "Young Adult"
    elif age < 60:
        return "Middle-Aged"
    else:
        return "Senior Citizen"

class AgeGenderApp:
    def __init__(self, window):
        self.window = window
        self.window.title("Real-time Age & Gender Prediction")
        self.window.geometry("1100x700")
        self.window.configure(bg='#e6f2ff')

        if os.path.exists("drdo_logo.png"):
            drdo_img = Image.open("drdo_logo.png")
            drdo_img = drdo_img.resize((100, 100))
            self.drdo_logo = ImageTk.PhotoImage(drdo_img)
            logo_label = tk.Label(window, image=self.drdo_logo, bg='#e6f2ff')
            logo_label.pack(pady=5)

        ttk.Label(window, text="DRDO - Age and Gender Prediction System", font=("Helvetica", 20, "bold"), background='#e6f2ff', foreground='#003366').pack(pady=10)

        frame = ttk.Frame(window)
        frame.pack()

        self.video_label = ttk.Label(frame)
        self.video_label.grid(row=0, column=0, padx=20)

        self.result_box = tk.Text(frame, width=40, height=25, font=("Courier", 12))
        self.result_box.grid(row=0, column=1, padx=10)

        btn_frame = ttk.Frame(window)
        btn_frame.pack(pady=10)

        self.start_btn = ttk.Button(btn_frame, text="Start Camera", command=self.start_camera)
        self.start_btn.grid(row=0, column=0, padx=10)

        self.stop_btn = ttk.Button(btn_frame, text="Stop Camera", command=self.stop_camera)
        self.stop_btn.grid(row=0, column=1, padx=10)

        self.cap = None
        self.running = False
        self.last_frame_time = time.time()

    def start_camera(self):
        if not self.running:
            self.running = True
            self.cap = cv2.VideoCapture(0)
            self.thread = threading.Thread(target=self.process_video)
            self.thread.start()

    def stop_camera(self):
        if self.running:
            self.running = False

    def process_video(self):
        while self.running:
            current_time = time.time()
            if current_time - self.last_frame_time < 0.1:  # Limit frame rate to 10 FPS
                time.sleep(0.01)
                continue
            self.last_frame_time = current_time

            ret, frame = self.cap.read()
            if not ret:
                continue

            frame = cv2.flip(frame, 1)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

            results = []
            for idx, (x, y, w, h) in enumerate(faces):
                face_img = frame[y:y+h, x:x+w]
                input_img = preprocess_face(face_img)

                gender_pred_prob, age_pred = model.predict(input_img, verbose=0)
                gender_label = "Female" if gender_pred_prob[0][0] > 0.5 else "Male"
                age_label = age_group(int(age_pred[0][0]))

                results.append(f"Person {idx + 1}: {gender_label}, {age_label}")
                label_text = f"{idx + 1}"
                cv2.putText(frame, label_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(rgb_frame)
            img = img.resize((640, 480))
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)

            self.result_box.delete("1.0", tk.END)
            if results:
                for line in results:
                    self.result_box.insert(tk.END, line + "\n")
            else:
                self.result_box.insert(tk.END, "No faces detected\n")

        if self.cap:
            self.cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    root = tk.Tk()
    app = AgeGenderApp(root)
    root.mainloop()
